# Parcial 1

In [1]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1746314465559_0001,pyspark,idle,Link,Link,None,


In [2]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1746314465559_0001,pyspark,idle,Link,Link,None,


In [3]:
# Spark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.window import Window

# Tipos de datos
from pyspark.sql.types import (
    StringType, FloatType, IntegerType, DateType, StructType, StructField
)

# Funciones de PySpark
from pyspark.sql.functions import (
    col, lit, lower, trim, regexp_replace, udf
)

# Otros
import unicodedata
from functools import reduce
import re

from pyspark.sql.functions import countDistinct


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1746314465559_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder \
    .appName("Profeco Parte A") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# bucket
NAME = 'thmrudolf' ##CAMBIAR AQUÍ SU NOMBRE.
BUCKET = f"s3://itam-analytics-{NAME}"
FOLDER = 'profeco'


# type of catalog
CATALOG_TYPE = 'basicos'

s3_path_parquet = f"{BUCKET}/{FOLDER}/parquet/"
df = spark.read.parquet(s3_path_parquet)
df.printSchema()
df.select("anio", "catalogo", "estado").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Parte A
En esta parte necesitarán levantar un cluster en AWS con Hadoop y Pyspark (Como lo hicimos en clase). Solo necesitan 1 cluster por equipo.

El nombre de tu cluster debe ser cluster_ + la mátricula (número de estudiante) más chica de los miembros del equipo. Por ejemplo: cluster_54903.
ETL con el Cluster.


Contesta las siguientes preguntas utilizando PySpark. Realiza el siguiente análisis (por año) y sobre todos los catálogos.

¿Cuántos catálogos diferentes tenemos?


In [ ]:
num_catalogos = df.select("catalogo").distinct().count()
print(f"Total de catálogos distintos: {num_catalogos}")


¿Cuáles son los 20 catálogos con más observaciones? Guarda la salida de este query en tu bucket de S3, lo necesitaremos más adelante.


In [ ]:
top_catalogos = df.groupBy("catalogo").count().orderBy(col("count").desc()).limit(20)

# Mostrar resultados
top_catalogos.show()

# Guardar la salida en S3
s3_path_top_20 = f"{BUCKET}/{FOLDER}/top_20_catalogos/"
top_catalogos.write.mode("overwrite").parquet(s3_path_top_20)


¿Tenemos datos de todos los estados del país? De no ser así, ¿cuáles faltan?


In [ ]:
# Obtener lista de estados únicos en los datos
estados_en_datos = df.select(col("estado")).distinct()
estados_en_datos.show(32)

In [ ]:
estados_mexico = [
    "aguascalientes", "baja california", "baja california sur", "campeche", "coahuila de zaragoza",
    "colima", "chiapas", "chihuahua", "durango", "guanajuato", "guerrero", "hidalgo",
    "jalisco", "ciudad de mexico", "estado de mexico", "michoacan de ocampo", "morelos", "nayarit", "nuevo leon", "oaxaca",
    "puebla", "queretaro", "quintana roo", "san luis potosi", "sinaloa", "sonora",
    "tabasco", "tamaulipas", "tlaxcala", "veracruz", "yucatan", "zacatecas"
]

# Convertir PySpark DataFrame a lista para comparación
estados_en_datos_lista = [row.estado for row in estados_en_datos.collect()]

# Encontrar estados faltantes
estados_faltantes = list(set(estados_mexico) - set(estados_en_datos_lista))

print(f"Estados faltantes en los datos: {estados_faltantes}")

¿Cuántas observaciones tenemos por estado?


In [ ]:
from pyspark.sql.functions import countDistinct

# Contar catálogos distintos por estado y año
catalogos_por_estado_anio = df.groupBy("estado", "anio") \
    .agg(countDistinct("catalogo").alias("num_catalogos"))

# Mostrar resultados
catalogos_por_estado_anio.show(10)

De cada estado obten: el número de catalogos diferentes por año, ¿ha aumentado el número de catálogos con el tiempo?
Utilizando Spark contesta las siguientes preguntas a partir del catálogo que le tocó a tu equipo. Recuerda trabajar en el archivo con los datos particionados de otra manera tus queries van a tardar mucho.

In [ ]:
from pyspark.sql.functions import countDistinct

# Obtener el número de catálogos distintos por estado y año
df_catalogos_por_estado_anio = df.groupBy("estado", "anio").agg(countDistinct("catalogo").alias("num_catalogos"))

# Mostrar resultados
df_catalogos_por_estado_anio.show()


In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

# Definir ventana por estado para comparar años anteriores
window_spec = Window.partitionBy("estado").orderBy("anio")

# Calcular la diferencia de catálogos con el año anterior
catalogos_por_estado_anio = catalogos_por_estado_anio.withColumn(
    "diferencia_anual",
    col("num_catalogos") - lag("num_catalogos", 1).over(window_spec)
)

catalogos_por_estado_anio.show(10)

In [ ]:
# Guardar los resultados en S3 para su posterior análisis

s3_path_estado_anio = f"{BUCKET}/{FOLDER}/catalogos_por_estado_anio/"
df_catalogos_por_estado_anio.write.mode("overwrite").parquet(s3_path_estado_anio)

print(f"Los resultados se han guardado en: {s3_path_estado_anio}")

¿Cuańtas marcas diferentes tiene tu categoría?

In [ ]:
df.show()

In [ ]:
catalogo_objetivo = "basicos"
df_filtered = df.filter(df["catalogo"] == catalogo_objetivo)


# Contar marcas distintas por categoría
num_marcas = df_filtered.select(countDistinct("marca").alias("num_marcas")).withColumn("catalogo", lit(catalogo_objetivo))



# Mostrar resultados

num_marcas.show()

¿Cuál es la marca con mayor precio? ¿En qué estado?


In [ ]:
df_max_precio = df.orderBy(col("precio").desc()).limit(1)#.withColumn("catalogo", lit(catalogo_objetivo))
df_max_precio.show()

¿Cuál es la marca con menor precio en CDMX? (en aquel entonces Distrito Federal)


In [ ]:
unique_values = df.select("estado").distinct().collect()
for row in unique_values:
    print(row["estado"])

In [ ]:
df_cdmx = df.filter(df['estado'] == 'ciudad de mexico')

df_cdmx_menor_precio_cdmx = df_cdmx.orderBy(col("precio").asc()).limit(1)
df_cdmx_menor_precio_cdmx.show()

¿Cuál es la marca con mayores observaciones?


In [ ]:
df_filtered_by_cat = df.filter(df['catalogo']=='basicos')
df_producto_mas_frecuente = df_filtered_by_cat.groupBy("producto").count().orderBy(col("count").desc()).limit(1)

# Mostrar resultados
df_producto_mas_frecuente.show()

df_marca_mas_frecuente = df_filtered_by_cat.groupBy("marca").count().orderBy(col("count").desc()).limit(1)

# Mostrar resultados
df_marca_mas_frecuente.show()


¿Cuáles son el top 5 de marcas con mayor precio en cada estado? ¿Son diferentes?


In [ ]:
from pyspark.sql.functions import col, row_number
#Calcular el precio máximo por marca y estado
df_precio_por_marca_estado = df.groupBy("estado", "marca", "precio").agg(col("precio").alias("max_precio"))

# Aplicar ventana para obtener el top 5 de marcas con mayor precio por estado
window_spec = Window.partitionBy("estado").orderBy(col("max_precio").desc())
df_top5 = df_precio_por_marca_estado.withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 5)

# Mostrar resultados
df_top5.select("estado", "marca", "max_precio").show()


¿Cuáles son el top 5 de marcas con menor precio en CDMX? (en aquel entonces Distrito Federal)


In [ ]:
#Calcular el precio máximo por marca y estado
df_cdmx = df.filter(df['estado']=='ciudad de mexico')
window_spec = Window.orderBy(col("precio").asc())
df_top5_cdmx = df_cdmx.withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 5)

# Mostrar resultados
df_top5_cdmx.select("producto", "estado", "precio").show()
df_top5_cdmx.select("marca", "estado", "precio").show()



¿Cuáles son el top 5 de marcas con mayores observaciones? ¿Se parecen a las de nivel por estado?


In [ ]:
# Obtener el conteo de observaciones por marca
df_top5_global = df.groupBy("marca").count().orderBy(col("count").desc()).limit(5)

# Mostrar resultados
df_top5_global.show()

# Obtener el conteo de observaciones por marca
df_top5_global_p = df.groupBy("producto").count().orderBy(col("count").desc()).limit(5)

# Mostrar resultados
df_top5_global_p.show()

In [ ]:
# Aplicar ventana para obtener el top 5 de cada estado (marca)
window_spec = Window.partitionBy("estado").orderBy(col("count").desc())

df_top5_estado = df.groupBy("estado", "marca").count() \
    .withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") <= 5)

# Mostrar resultados
df_top5_estado.show()

# Aplicar ventana para obtener el top 5 de cada estado (producto)
df_top5_estado_p = df.groupBy("estado", "producto").count() \
    .withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") <= 5)

# Mostrar resultados
df_top5_estado_p.show()



 ¿Se parecen a las de nivel por estado?

In [ ]:
from pyspark.sql.functions import collect_list

# Obtener lista de marcas en el top global
marcas_top_global = [row["marca"] for row in df_top5_global.collect()]

# Agrupar marcas del top 5 por estado y compararlas con el top global
df_comparacion = df_top5_estado.groupBy("estado").agg(collect_list("marca").alias("top_5_marcas"))

df_comparacion.show(truncate=False)

# Obtener lista de marcas en el top global
producto_top_global = [row["producto"] for row in df_top5_global_p.collect()]

# Agrupar marcas del top 5 por estado y compararlas con el top global
df_comparacion_p = df_top5_estado_p.groupBy("estado").agg(collect_list("producto").alias("top_5_productos"))

df_comparacion.show(truncate=False)

¿Ha dejado de existir alguna marca durante los años que tienes? ¿Cuál? ¿Cuándo desapareció?


In [ ]:

# Obtener marcas únicas por año
df_marcas_por_anio = df.select("anio", "marca").distinct()

# Obtener todas las marcas que existieron alguna vez
marcas_existentes = df_marcas_por_anio.select("marca").distinct()

# Obtener la última aparición de cada marca
df_ultima_aparicion = df_marcas_por_anio.groupBy("marca").agg({"anio": "max"}).withColumnRenamed("max(anio)", "ultimo_anio")

# Comparar con las marcas actuales (último año presente en los datos)
ultimo_anio = df.select("anio").distinct().orderBy("anio", ascending=False).limit(1).collect()[0]["anio"]
df_marcas_desaparecidas = df_ultima_aparicion.filter(df_ultima_aparicion["ultimo_anio"] < ultimo_anio)

# Mostrar resultados
df_marcas_desaparecidas.show()

Genera una gráfica de serie de tiempo por estado para la marca con mayor precio -en todos los años-, donde el eje equis es el año y el eje ye es el precio máximo.
Nota: Recuerden descargar del cluster su análisis en Jupyter, de otra manera se borrará.

Hint: Guarda tus consultas en archivos que puedas guardar en S3 y luego leer desde Pandas o RStudio, para hacer tus gráficas o cuadros compartivos.

In [ ]:
spark._sc.install_pypi_package("matplotlib", "https://pypi.org/simple")

In [ ]:
import matplotlib.pyplot as plt

# Inicializar Spark Session
spark = SparkSession.builder.appName("Time Series Analysis").getOrCreate()

# Cargar datos desde S3
df = spark.read.parquet("s3a://tu-bucket/datos.parquet")

# Obtener la marca con el mayor precio en todos los años
df_max_precio_marca = df.groupBy("estado", "anio", "marca").agg(col("precio").alias("max_precio")) \
    .orderBy(col("max_precio").desc()).dropDuplicates(["anio", "estado"])

# Convertir a Pandas para graficar
df_pandas = df_max_precio_marca.toPandas()

# Crear gráfica de serie de tiempo
plt.figure(figsize=(12, 6))
for estado in df_pandas["estado"].unique():
    data_estado = df_pandas[df_pandas["estado"] == estado]
    plt.plot(data_estado["anio"], data_estado["max_precio"], marker="o", label=estado)

plt.xlabel("Año")
plt.ylabel("Precio Máximo")
plt.title("Evolución del Precio Máximo por Estado (Marca Más Cara)")
plt.legend()
plt.grid(True)
plt.show()